In [1]:
from data_prep import *
from charts import *
import pandas as pd

game_df = team_sheets()
players_df = players(game_df)
players_agg_df = players_agg(players_df)
lineouts_df = lineouts()
pitchero_df = pitchero_stats()
set_piece_df = set_piece_results()
analysis = game_stats()

/Users/samlindsay/Documents/Projects/Personal/RugbyResults/env/lib/python3.10/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


# Season Summaries

In [ ]:
seasons = [
    "2021/22", 
    "2022/23", 
    "2023/24", 
    "2024/25"
]

for season in seasons:
    print(season)

    # Starts (by Position)
    plot_starts_by_position(1, season=season, fb_only=False, df=players_df).save(f"Charts/{season.replace('/','-')}/1s-squad-position.html")
    plot_starts_by_position(2, season=season, fb_only=False, df=players_df).save(f"Charts/{season.replace('/','-')}/2s-squad-position.html")
    # Appearances
    plot_games_by_player(0, min=10, season=season, df=players_df).save(f"Charts/{season.replace('/','-')}/appearances.html")
    plot_games_by_player(1, min=5, season=season, df=players_df).save(f"Charts/{season.replace('/','-')}/1s-appearances.html")
    plot_games_by_player(2, min=5, season=season, df=players_df).save(f"Charts/{season.replace('/','-')}/2s-appearances.html")
    # Points Scorers
    points_scorers_chart(1, season, df=pitchero_df).save(f"Charts/{season.replace('/','-')}/1s-points.html")
    points_scorers_chart(2, season, df=pitchero_df).save(f"Charts/{season.replace('/','-')}/2s-points.html")
    # Captains
    captains_chart(season, df=game_df).save(f"Charts/{season.replace('/','-')}/captains.html")
    # Cards
    card_chart(0, season, df=pitchero_df).save(f"Charts/{season.replace('/','-')}/cards.html")
    # Resultsx
    results_chart(0, season, df=game_df).save(f"Charts/{season.replace('/','-')}/results.html")
    # results_chart(1, season, df=game_df).save(f"Charts/{season.replace('/','-')}/1s-results.html")
    # results_chart(2, season, df=game_df).save(f"Charts/{season.replace('/','-')}/2s-results.html")
    
    # Set Piece
    set_piece_h2h_charts(1, season, df=set_piece_df).save(f"Charts/{season.replace('/','-')}/1s-set-piece.html")
    set_piece_h2h_charts(2, season, df=set_piece_df).save(f"Charts/{season.replace('/','-')}/2s-set-piece.html")

    # # Lineouts
    lineout_chart(1, season, df=lineouts_df).save(f"Charts/{season.replace('/','-')}/1s-lineouts.html")
    if season != "2021/22":
        lineout_chart(2, season, df=lineouts_df).save(f"Charts/{season.replace('/','-')}/2s-lineouts.html")

2021/22


/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:812: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "Cards"] = df.loc[:,"YC"] + df.loc[:,"RC"]
/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:891: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,"loser"] = df.apply(lambda x: x["PF"] if x["Result"] == "L" else x["PA"], axis=1)
/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:892: SettingWithCopyWarning: 
A value is trying to be set o

2022/23


/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:812: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "Cards"] = df.loc[:,"YC"] + df.loc[:,"RC"]
/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:891: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,"loser"] = df.apply(lambda x: x["PF"] if x["Result"] == "L" else x["PA"], axis=1)
/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:892: SettingWithCopyWarning: 
A value is trying to be set o

2023/24


/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:812: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "Cards"] = df.loc[:,"YC"] + df.loc[:,"RC"]
/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:891: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,"loser"] = df.apply(lambda x: x["PF"] if x["Result"] == "L" else x["PA"], axis=1)
/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:892: SettingWithCopyWarning: 
A value is trying to be set o

2024/25


/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:812: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "Cards"] = df.loc[:,"YC"] + df.loc[:,"RC"]
/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:891: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,"loser"] = df.apply(lambda x: x["PF"] if x["Result"] == "L" else x["PA"], axis=1)
/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:892: SettingWithCopyWarning: 
A value is trying to be set o

# Trends


In [2]:
plot_games_by_player(0, min=10, season=None, agg=True, df=players_df).save("Charts/by-season/appearances.html")
captains_chart(season=None, df=game_df).save("Charts/by-season/captains.html")
card_chart(season=None, df=pitchero_df).save("Charts/by-season/cards.html")
points_scorers_chart(squad=0, season=None, df=pitchero_df).save("Charts/by-season/points.html")
results_chart(squad=0, season=None, df=game_df).save("Charts/by-season/results.html")

# Set Piece

In [4]:
set_piece_h2h_charts(1, season=None, df=set_piece_df).save("Charts/by-season/1s-set-piece-h2h.html")
set_piece_h2h_charts(2, season=None, df=set_piece_df).save("Charts/by-season/2s-set-piece-h2h.html")

# 1st XV Game Analysis

In [8]:
game_stats_charts(analysis).save("Charts/by-season/game-stats.html")

/Users/samlindsay/Documents/Projects/Personal/RugbyResults/env/lib/python3.10/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:1427: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  penalties.loc[:,"stack_order"] = penalties.loc[:,"Metric"].apply(lambda x: penalty_cols.index(x))
/Users/samlindsay/Documents/Projects/Personal/RugbyResults/charts.py:1429: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

In [2]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

pitchero_df

,Player,Season,Squad,TotalGames,Player_join,A,T,Con,PK,DG,YC,RC,Points,PPG,Tries,Cons,Pens
0,Dave Bridges,2022/23,1st,14,D Bridges,15.0,4.0,0.0,0.0,0.0,1.0,0.0,20.0,1.333333,20.0,0.0,0.0
1,Josh Brimecombe,2022/23,1st,15,J Brimecombe,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,Guy Collins,2024/25,1st,13,G Collins,13.0,2.0,0.0,0.0,0.0,1.0,0.0,10.0,0.769231,10.0,0.0,0.0
3,George Naylor,2023/24,2nd,9,G Naylor,8.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,0.625000,5.0,0.0,0.0
4,Michael Presland,2024/25,2nd,8,M Presland,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,Josh Weaver,2023/24,1st,1,J Weaver,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
519,Colm Geraghty,2024/25,2nd,1,C Geraghty,1.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,5.000000,5.0,0.0,0.0
520,Connor Williams,2024/25,2nd,1,C Williams,1.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,5.000000,5.0,0.0,0.0
521,Darren Gwynn,2023/24,2nd,1,D Gwynn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
def points_scorers_chart(squad=1, season="2024/25", df=None):

    if df is None:
        df = pitchero_stats()
    if squad != 0:
        df = df[df["Squad"] == ("1st" if squad == 1 else "2nd")]
    if season:
        df = df[df["Season"] == season]
    
    scorers = df[df["Points"] > 0]

    scorers = scorers.drop("Points", axis=1)
    scorers = scorers.melt(
        id_vars=[c for c in scorers.columns if c not in ["Tries", "Pens", "Cons"]], 
        var_name="Type", 
        value_name="Points"
    )

    selection = alt.selection_point(fields=['Type'], bind='legend')

    chart = (
        alt.Chart(scorers)
        .add_params(selection)
        .transform_filter(selection)
        .transform_filter("datum.Points > 0")
        .transform_calculate(label="if(datum.T>0, datum.T + 'T ','') + if(datum.PK>0, datum.PK + 'P ', '') + if(datum.Con>0, datum.Con + 'C ', '')")
        .mark_bar()
        .encode(
            x=alt.X("sum(Points):Q", axis=alt.Axis(orient="top", title="Points")),
            y=alt.Y(
                "Player:N", 
                sort=alt.EncodingSortField(field="sortfield", order="descending"), 
                title=None
            ),
            tooltip=[
                alt.Tooltip("Player:N", title=" "), 
                alt.Tooltip("A:Q", title="Games"),
                alt.Tooltip("label:N", title="Scores"),
                alt.Tooltip("sortfield:Q", title="Points")
            ],
        )
        .transform_joinaggregate(
            sortfield="sum(Points)",    
            groupby=["T", "Pen", "Con", "Player", "Season", "Squad"]
        )
        .properties(width=400 if season else 200, height=alt.Step(16))
    )
    
    text = (
        chart    
        .mark_text(align="left", dx=5, color="black")
        .encode(
            y=alt.Y(
                "Player:N", 
                sort=alt.EncodingSortField(field="sortfield", order="descending"), 
                title=None
            ),
            x=alt.X("sortfield:Q", axis=alt.Axis(orient="bottom", title="Points")),
            text=alt.Text("label:N")
        )
    )

    chart = chart.encode(
        color=alt.Color(
                "Type:N", 
                legend=alt.Legend(
                    title="Click to filter",
                    titleOrient="left",
                    orient="bottom",
                ), 
                scale=alt.Scale(domain=['Tries', 'Pens', 'Cons'], range=["#202947", "#981515", "#146f14"])
            ),
        order=alt.Order("Type:N", sort="descending"),
    )


    chart = (
        (chart + text)
        # chart
        .facet(
            row=alt.Row(
                "Squad:N", 
                header=alt.Header(title=None, labelExpr="datum.value + ' XV'", labelFontSize=36) if squad == 0 else None
            ),
            column=alt.Column(
                "Season:O", 
                header=alt.Header(title=None, labelFontSize=28) if season is None else None, 
            ),
            spacing=20,
            align="each"
        )
        .properties(
            title=alt.Title(
                text=("1st XV " if squad==1 else "2nd XV " if squad==2 else "") + "Points Scorers",
                subtitle="According to Pitchero data"
            )
        )
        .resolve_scale(y="independent")
    )

    return chart

points_scorers_chart(0, season=None, df=pitchero_df).save("Charts/by-season/points.html")

# Team Sheets

In [ ]:
def team_sheet_chart(
        squad=1, 
        names=None, 
        captain=None, 
        vc=None, 
        opposition=None, 
        home=True, 
        competition="Counties 1 Sussex",
        season="2023/24"
    ):

    if names is None:
        df = team_sheets(squad=1) 

        # Last row as dict
        team = df.iloc[-1].to_dict()


        label = f'{"1st" if squad==1 else "2nd"} XV vs {team["Opposition"]}({team["Home/Away"]})'
        captain = team["Captain"]
        vc = team["VC"]
        season = team["Season"]
        competition = team["Competition"]

        # Keep keys that can be converted to integers
        team = {int(k): v for k, v in team.items() if k.isnumeric() and v}

        # Convert team to dataframe with Number and Player columns
        team = pd.DataFrame(team.items(), columns=["Number", "Player"])

    else:
        label = f'{"1st" if squad==1 else "2nd"} XV vs {opposition} ({"H" if home else "A"})'

        # Convert names to Player column of a dataframe with Number column (1-len(names))
        team = pd.DataFrame({"Player": names, "Number": range(1, len(names)+1)})

    coords = pd.DataFrame([
                {"n": 1, "x": 10, "y": 81},
                {"n": 2, "x": 25, "y": 81},
                {"n": 3, "x": 40, "y": 81},
                {"n": 4, "x": 18, "y": 69},
                {"n": 5, "x": 32, "y": 69},
                {"n": 6, "x": 6, "y": 61},
                {"n": 7, "x": 44, "y": 61},
                {"n": 8, "x": 25, "y": 56},
                {"n": 9, "x": 20, "y": 42},
                {"n": 10, "x": 38, "y": 36},
                {"n": 11, "x": 8, "y": 18},
                {"n": 12, "x": 56, "y": 30},
                {"n": 13, "x": 74, "y": 24},
                {"n": 14, "x": 92, "y": 18},
                {"n": 15, "x": 50, "y": 10},
                {"n": 16, "x": 80, "y": 82},
                {"n": 17, "x": 80, "y": 74},
                {"n": 18, "x": 80, "y": 66},
                {"n": 19, "x": 80, "y": 58},
                {"n": 20, "x": 80, "y": 50},
                {"n": 21, "x": 80, "y": 42},
                {"n": 22, "x": 80, "y": 34},
                {"n": 23, "x": 80, "y": 26},
            ])
    team = team.merge(coords, left_on="Number", right_on="n", how="inner").drop(columns="n")

    # Add captain (C) and vice captain (VC) else None
    team["Captain"] = team["Player"].apply(lambda x: "C" if x == captain else "VC" if x == vc else None)

    team["Player"] = team["Player"].str.split(" ")

    team.to_dict(orient="records")

    with open("team-sheet-lineup.json") as f:
        chart = json.load(f)
    chart["data"]["values"] = team.to_dict(orient="records")
    chart["title"]["text"] = label
    chart["title"]["subtitle"] = f"{season} - {competition}"

    n_replacements = len(team) - 15
    
    y = 126 + (n_replacements * 64)
    chart["layer"][0]["mark"]["y2"] = y
    # return chart
    return alt.Chart.from_dict(chart)

In [96]:
pitchero_df[pitchero_df["Player"]=="Sam Lindsay-McCall"]

,Player,Season,Squad,TotalGames,Player_join,A,T,Con,PK,DG,YC,RC,Points,PPG,Tries,Cons,Pens,Cards
9,Sam Lindsay-McCall,2023/24,1st,23,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,Sam Lindsay-McCall,2021/22,2nd,2,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,Sam Lindsay-McCall,2024/25,2nd,1,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,Sam Lindsay-McCall,2021/22,1st,15,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,Sam Lindsay-McCall,2024/25,1st,15,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
270,Sam Lindsay-McCall,2022/23,2nd,1,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
470,Sam Lindsay-McCall,2022/23,1st,15,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
pitchero_df[pitchero_df["Player_join"]=="S Lindsay"].sort_values(["Season","Squad"])

,Player,Season,Squad,TotalGames,Player_join,A,T,Con,PK,DG,YC,RC,Points,PPG,Tries,Cons,Pens
464,Sam Lindsay-McCall,2021/22,1st,15,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
348,Sam Lindsay-McCall,2021/22,2nd,2,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,Sam Lindsay-McCall,2022/23,1st,15,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
405,Sam Lindsay-McCall,2022/23,2nd,1,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
346,Sam Lindsay-McCall,2023/24,1st,23,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
466,Sam Lindsay-McCall,2024/25,1st,15,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388,Sam Lindsay-McCall,2024/25,2nd,1,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
